# Content Mining Models

## Table of Contents
##### [1. Data Preprocessing and Imports](#preprocessing)
##### [2. Models](#models)
###### [2.1. Cosine Similarity](#cosine)
###### [2.2. LSI Model](#lsi)
###### [2.3. Mixture Model](#mixture)
##### [3. Interpretation and Evaluation](#interpretation_evaluation)

<a id='preprocessing'></a>
## 1. Data Preprocessing and Imports

In [2]:
import pandas as pd

In [1]:
# Import functions module
%run functions.py

In [3]:
interactions_data = pd.read_csv(
        'C:/Users/d067795/OneDrive - SAP SE/Documents/Master/Semester 2/Web Mining/Project/RAW_interactions.csv')
recipes_data = pd.read_csv(
        'C:/Users/d067795/OneDrive - SAP SE/Documents/Master/Semester 2/Web Mining/Project/RAW_recipes.csv')

In [4]:
# Rename columns to more explanatory names
recipes_data.rename(columns={"id": "recipe_id"}, inplace=True)
interactions_data.rename(columns={"num_interactions": "date", "avg_rating": "rating"}, inplace=True)

# Fill nan
recipes_data.fillna("", inplace=True)
interactions_data.fillna("", inplace=True)

In [5]:
# Preprocess ingredients and save as String
for index, row in recipes_data.iterrows():
    ingredientlist = row['ingredients']
    ingredientlist = row['ingredients'].replace('[', '').replace(', ', '').replace(']', '').replace('and', '\'').split("\'")
    ingredientlist = list(filter(None, ingredientlist))
    ingredientlistString = ""
    for i in ingredientlist:
        ingredientlistString = ingredientlistString + i
    recipes_data.at[index, 'ingredients'] = ingredientlistString

In [ ]:
# Extract keywords for free text features
get_keywords(recipes_data, "steps", "steps_keywords")
get_keywords(recipes_data, "description", "description_keywords")
get_keywords(interactions_data, "review", "review_keywords")

In [ ]:
# Merge datasets
#Average ratings
num_interactions = interactions_data.groupby("recipe_id")["date"].count()

#only consider the ratings (>0) into the mean, not the reviews w/o ratings
mean_ratings = interactions_data[interactions_data["rating"]!=0].groupby("recipe_id")["rating"].mean()

df_rmerged = recipes_data.join(num_interactions, how="left", on="recipe_id").join(mean_ratings, how="left", on="recipe_id")

<a id='models'></a>
## 2. Models

<a id='cosine'></a>
### 2.1. Cosine Similarity

<a id='lsi'></a>
### 2.2. LSI Model

<a id='mixture'></a>
### 2.3. Mixture Model

In [ ]:
lmbda = 0.5
processed = df_rmerged["recipe_id", "review_keywords"]

cos_sim = get_cos_sim_matrix(processed)
business_sub = business[['business_id', 'latitude', 'longitude', 'stars', 'review_count', 'is_open']]
business_processed = business_sub[business_sub['business_id'].isin(processed['business_id'])]\
                                                             .set_index('business_id')
scaler = StandardScaler()
X = scaler.fit_transform(business_processed)
eucl_dis = euclidean_distances(X,X)

eucl_sim = 1/np.exp(eucl_dis)  mixed_sim = np.add(cos_sim*lmbda,eucl_sim*(1-lmbda)) # assume equally weighted

<a id='interpretation_evaluation'></a>
## 3. Interpretation and Evaluation